## ESG Research 
### Yfinance, Pandas, NumPy


# Installation
* pip install pandas
* pip install numpy
* pip install yfinance | pip3 install yfinance
* pip install git+https://github.com/rodrigobercini/yfinance.git


In [115]:
#Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import Series, DataFrame
from pandas.tseries import offsets
import csv

Download S&P 500 tickers from csv file: *tickers.csv*

In [58]:
df = pd.read_csv ('tickers.csv')
df.head(5) #Shows first 5 tickers in list 

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


Get a random ticker from the dataset using *sample()*

In [59]:
tic = df.sample(n=1)
ticker = tic.loc[tic.index[0],'Symbol']
print(tic)

    Symbol              Name      Sector
285    LNC  Lincoln National  Financials


In [60]:
ticker = yf.Ticker(ticker)

In [ ]:
# -- What information is included in the dictionary? --
for k in ticker.info.keys():
    print(k)

In [67]:
ticker = yf.Ticker('XOM')
y = ticker.sustainability 

Now, we want to write the data into the csv file to build up the database

In [142]:
from csv import writer
  
# The data assigned to the list 
#list_data=['04','Smih','Sciee']
  

with open('data.csv', 'a', newline='') as f_object:  
    writer_object = writer(f_object) # Pass the CSV  file object to the writer() function
    writer_object.writerow(list_data) # Pass the data in the list as an argument into the writerow() function
    f_object.close()


#
